In [78]:
from pydub import AudioSegment
from scipy import signal
from sklearn.utils import shuffle
import re
import pandas as pd
import numpy as np
from glob import iglob
import statistics as stats
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
from librosa import display
from sklearn.metrics import confusion_matrix
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Embedding, Activation
from keras.layers import Conv1D, Conv2D, MaxPooling2D, ConvLSTM2D, LSTM, Bidirectional, GRU, TimeDistributed
from keras.optimizers import sgd, Adam
from keras.engine.input_layer import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
import keras.backend as K
from keras import regularizers
from keras.layers import Lambda
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
import tensorflow as tf
import itertools
import math
import time
import os
import json

In [30]:
def segment_audio(ds_total, transcript_directory, audio_directory, audio_out):
    input_array = []
    person_id_list = []
    level_array = []
    
    filenames = os.listdir(transcript_directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(transcript_directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            """
            person_id = int(m.group(1))            
            person_audio = AudioSegment.from_wav(audio_directory + str(person_id) + '_AUDIO.wav')
            lines = len(transcript)
            combined_sounds = AudioSegment.empty()
            for i in range(0, lines):
                row = transcript.iloc[i]
                if row["speaker"] == "Participant":
                    t1 = int(float(row["start_time"]) * 1000)
                    t2 = int(float(row["stop_time"]) * 1000)
                    newAudio = person_audio[t1:t2]
                    combined_sounds += newAudio
                    
            combined_sounds.export(audio_out + str(person_id) + '.wav', format="wav")
            """
            w = AudioSegment.from_wav(audio_out + str(person_id) + '.wav')
            input_array.append(w)
            person_id_list.append(person_id)
            level_array.append(ds_total.loc[ds_total["Participant_ID"] == person_id, "level"].item())
    return input_array, person_id_list, level_array

In [31]:
num_classes = 5
data_path = "F:/DIAC-WOZ/transcripts/"
audio_path = "F:/DIAC-WOZ/audio/"

In [32]:
def load_avec_dataset_file(path, score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist()
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score']]
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [33]:
train = load_avec_dataset_file('F:/DIAC-WOZ/train_split_Depression_AVEC2017.csv', 'PHQ8_Score')
dev = load_avec_dataset_file('F:/DIAC-WOZ/dev_split_Depression_AVEC2017.csv', 'PHQ8_Score')
test = load_avec_dataset_file('F:/DIAC-WOZ/full_test_split.csv', 'PHQ_Score')
print("Size: train= {}, dev= {}, test= {}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test= 47


,Participant_ID,level,cat_level,PHQ8_Score
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2


In [34]:
ds_total = pd.concat([train,dev,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [35]:
ds_total.to_csv(data_path + 'ds_total.csv', sep='\t')
print("File was created")

File was created


In [36]:
input_array, person_id_list, level_array = segment_audio(ds_total, data_path , audio_path, "F:/DIAC-WOZ/audio_cut/")

In [47]:
window_size = 100 * 1000
windows_path = "F:/DIAC-WOZ/audio_windows/"
for audio_file, person_id in zip(input_array, person_id_list):
    no_of_samples = len(audio_file)/window_size
    start = 0
    end = window_size
    for sample_number in range(int(no_of_samples)):
        audio_window = AudioSegment.empty()
        audio_window = audio_file[start:end]
        start = start + window_size
        end = end + window_size
        audio_window.export(windows_path + str(person_id) +'_'+ str(sample_number) + '.wav', format="wav")
    
    if len(audio_file) < end and end-len(audio_file)<=90:
        audio_window = AudioSegment.empty()
        audio_window = audio_file[start:len(audio_file)]
        np.concatenate((audio_window, np.zeros(shape=(end - len(audio_file), 1))))
        audio_window.export(windows_path + str(person_id) +'_'+ str(sample_number) + '.wav', format="wav")
    
    print(person_id)
    

300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
395
396
397
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492


In [63]:
train_audio_files= {}
test_audio_files= {}
dev_audio_files= {}

windows_path = "F:/DIAC-WOZ/audio_windows/"

for person_id, level in zip(train['Participant_ID'], train['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(AudioSegment.from_wav(wav_filename))
    print(int(person_id))
    train_audio_files[int(person_id)] = audio_content

303
304
305
310
312
313
315
316
317
318
319
320
321
322
324
325
326
327
328
330
333
336
338
339
340
341
343
344
345
347
348
350
351
352
353
355
356
357
358
360
362
363
364
366
368
369
370
371
372
374
375
376
379
380
383
385
386
391
392
393
397
400
401
402
409
412
414
415
416
419
423
425
426
427
428
429
430
433
434
437
441
443
444
445
446
447
448
449
454
455
456
457
459
463
464
468
471
473
474
475
478
479
485
486
487
488
491


In [64]:
for person_id, level in zip(dev['Participant_ID'], dev['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(AudioSegment.from_wav(wav_filename))
    dev_audio_files[int(person_id)] = audio_content

In [65]:
for person_id, level in zip(test['Participant_ID'], test['level']):
    audio_content = []
    for i, wav_filename in enumerate(iglob(os.path.join(windows_path, str(int(person_id)) + "**.wav"))):
        audio_content.append(AudioSegment.from_wav(wav_filename))
    test_audio_files[int(person_id)] = audio_content

In [72]:
AUDIO_LENGTH = 
def m5(num_classes=5):
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [79]:
model = m5(num_classes=5)
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

Using Model M5


In [86]:

x_tr = []
y_tr = []

x_te = []
y_te = []

train_y = {}
for person_id, level in zip(train['Participant_ID'], train['level']):
    train_y[int(person_id)] = level
    

for item in train_audio_files.items():
    print(item[0])
    for file in item[1]:
        x_tr.append(file.get_array_of_samples())
        y_tr.append(train_y[item[0]])
        break
    break

print(x_tr)
print(y_tr)


# reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
# batch_size = 128
# model.fit(x=x_tr,
#           y=y_tr,
#           batch_size=batch_size,
#           epochs=400,
#           verbose=1,
#           shuffle=True,
#           validation_data=(x_te, y_te),
#           callbacks=[reduce_lr])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

